In [ ]:
import pandas as pd

In [ ]:

customers_df = pd.read_csv('../data/hm/customers.csv')
article_df = pd.read_csv('../data/hm/articles.csv', dtype={'article_id': str})
transactions_df = pd.read_csv('../data/hm/transactions_train.csv', dtype={'article_id': str})

In [ ]:
article_df = article_df[['article_id', 'prod_name', 'product_type_name', 'colour_group_name', 'section_name']]
customers_df = customers_df[['customer_id', 'age']]

In [ ]:
df = pd.merge(transactions_df, article_df, how='inner', on='article_id')

customer_count = df.customer_id.value_counts()
valid_customers = customer_count[5000:15000]
valid_customer_ids = set(valid_customers.index)

valid_customer_mask = df.customer_id.isin(valid_customer_ids)

a = df.article_id.value_counts()
too_popular_articles = set(a[:50].index)
too_unpopular_articles = set(a[a < 20].index)

exclude_article_mask = df.article_id.isin(too_popular_articles | too_unpopular_articles)

mask = (valid_customer_mask & (~exclude_article_mask))

filtered_df = df[mask]

In [ ]:
from tqdm import tqdm
tqdm.pandas()

raw_sequences = []
customer_ids = []
seq_labels = None

for customer_id, personal_df in tqdm(filtered_df.groupby('customer_id')):
    raw_sequences.append(list(personal_df.article_id.values))
    customer_ids.append(customer_id)

In [ ]:
filtered_articles = filtered_df.article_id.unique()

In [ ]:
filtered_articles_df = article_df[article_df.article_id.isin(set(filtered_articles))]
filtered_articles_df = filtered_articles_df.set_index('article_id')
filtered_articles_df.to_csv('../data/hm/items.csv')

In [ ]:
item_list = []

for index, row in filtered_articles_df.iterrows():
    item_name = row.prod_name + ' - ' + row.product_type_name
    item_list.append(item_name)

In [ ]:
raw_sequences = [' '.join(sequence) for sequence in raw_sequences]

In [ ]:
sequence_df = pd.DataFrame(raw_sequences, columns=['sequence'])
sequence_df['customer_id'] = customer_ids
sequence_df.to_csv('../data/hm/filtered_purchase_history.csv', index=False)

In [ ]:
items = pd.read_csv("../data/hm/items.csv", dtype={"article_id": str}, index_col='article_id')

In [ ]:
customers_df.index = customers_df.customer_id
customers_df = customers_df.drop("customer_id", axis=1)

In [ ]:
customers_df.to_csv('../data/hm/customers.csv')

In [ ]:
pd.read_csv('../data/hm/filtered_purchase_history.csv')